# Modelo de Classificação - Variável Target Binária

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problema de Gestão

O dataset contém características dos clientes de um banco. O nosso objetivo é desenvolver um modelo para predição de quando o cliente deixará o banco. A variável target se chama " Exited ", é binária : 0 quando o cliente ainda permanece no banco e 1 quando ele deixou de ser cliente.

# Base

In [ ]:
# Importando a base

df = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')
                 
df.shape

In [ ]:
# Visualizando os dados

df.head()

In [ ]:
# Visualizando tipos de dados

df.info()

# Variáveis Object

In [ ]:
# Análise de valores únicos das variáveis object

print('Surname: ',df['Surname'].nunique(),'\nGeography: ',df['Geography'].nunique(),'\nGender: ',df['Gender'].nunique())

In [ ]:
# Drop de variáveis com alta cardinalidade ou variáveis de contagem
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'],inplace=True)
df.head()

In [ ]:
# Visualização das variaveis no dataset
ax = df.hist(bins=25, grid=False, figsize=(18,18), color='#1DB954', zorder=2, rwidth=0.9)

In [ ]:
# Verificando a feature Estimated Salary
df.EstimatedSalary.describe()

In [ ]:
# Boxplot Estimated Salary

plt.figure(figsize=(8,8))
ax = sns.boxplot(y=df["EstimatedSalary"],color='#1DB954')
plt.title("Estimated Salary")
plt.show()

In [ ]:
# Criar variaveis Dummy para Gender e Geography

data = pd.concat([pd.get_dummies(df[['Gender', 'Geography']]), df[['CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary','Exited']]], axis=1)

In [ ]:
# Visualizando dados de forma transposta
data.head().T

# Random Forest Classifier

In [ ]:
# Treinamento do modelo

# Separando o dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(data, test_size=0.20, random_state=42)

# Separando treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# definindo colunas de entrada
feats = [c for c in data.columns if c not in ['Exited']]

feats

In [ ]:
# treinar o modelo

# Importando o modelo
from sklearn.ensemble import RandomForestClassifier

# Instanciar o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# treinar o modelo RandomForestClassifier
rf.fit(train[feats], train['Exited'])

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feats])

preds_val

# Métricas do modelo Random Forest Classifier

In [ ]:
# Avaliando o desempenho do modelo

# Importando a metrica
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import scikitplot as skplt

In [ ]:
# Medindo a acurácia nos dados de teste
preds_test = rf.predict(test[feats])

print("Acurácia:",accuracy_score(test['Exited'], preds_test))

In [ ]:
# Matriz de Confusão com os dados de teste
print("Matriz de Confusão:",confusion_matrix(test['Exited'], preds_test))

In [ ]:
# Matriz de Confusão com os dados de teste de forma gráfica
skplt.metrics.plot_confusion_matrix(test['Exited'], preds_test)

In [ ]:
# Olhando para variável target no dataset
test['Exited'].value_counts(normalize=True)

In [ ]:
test['Exited'].value_counts()

Dos 393 clientes que deixaram o banco, o nosso modelo conseguiu detectar 179 ( na partição teste )

# Comparação de modelos de classificação - Cross Val Score

In [ ]:
# Importando outros modelos

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def find_best(x,y,cv):
    
    ## instanciando modelos
    log = LogisticRegression()
    dt = DecisionTreeClassifier()
    
    ## gerando lista de modelos
    list_of_models = [log,dt,rf]
    
    ## gerando lista de scores
    dc_of_scores = {}
    
    ## iterando sobre modelos e executando a validação
    for idx,model in enumerate(list_of_models):
        dc_of_scores[idx] = cross_val_score(model,x,y,cv=cv,scoring='precision').mean()
    
    best_model = max(dc_of_scores,key=dc_of_scores.get)
    
    print('xxx Encontrando melhor modelo xxx\n')
    if best_model == 0:
        print(f'Melhor modelo: LogisticRegression\nPrecisão : {dc_of_scores[0]}')
    
    elif best_model == 1:
        print(f'Melhor modelo: DecisionTreeClassifier\nPrecisão : {dc_of_scores[1]}')
        
    elif best_model == 2:
        print(f'Melhor modelo: RandomForestClassifier\nPrecisão : {dc_of_scores[2]}')

    print(f'\nLogisticRegression\nPrecisão : {dc_of_scores[0]}')
    print(f'DecisionTreeClassifier\nPrecisão : {dc_of_scores[1]}')
    print(f'RandomForestClassifier\nPrecisão : {dc_of_scores[2]}')

In [ ]:
find_best(data[feats],data['Exited'],10)

No caso desse problema de gestão, a métrica usada para avialar qual melhor modelo foi PRECISÃO.
Por quê? Porque essa métrica mostra quão bem o modelo consegue captar os verdadeiros positivos. 
O objetivo é encontrar quem está propenso a sair do banco (Exited == 1). 
A acurácia mede uma perfomance geral. Nesse caso, foi necessário analisar precisão também.

Para seleção de melhor modelo foi usado o cross_val_score, que mostrou que o Random Forest Classifier é que possui a maior precisão entre Random Forest, Decision Tree e Logistic Regression.